In [10]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install flask_login

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask_cors

Note: you may need to restart the kernel to use updated packages.


In [ ]:
############# Flask Modules Setup ##############

from flask import Flask, render_template, url_for, redirect, request, make_response, Response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
from flask_cors import CORS

import pymysql

######### Initialize Flask App ##########

app = Flask(__name__)
CORS(app)

######### MySQL Database Setup ##########

connection = pymysql.connect(host="localhost",
                     user="root",   
                     passwd="root", 
                     db="comm5940",   
                     charset='utf8mb4')     

######### Login Manager Setup ##########

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"


@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin): 
    def __init__(self,id):
        self.id = id

############ Web Page Routes Setup ###############
    
@app.route("/")
@login_required
def home():
    return render_template('index.html')

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "1155129577@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Ellie's pages. Your login has been granted."
        return make_response(message, 200, )
    else:
        message = 'Wrong password!'
        return make_response( messagte,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "1155129577@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Ellie's pages. Your login has been granted."
        return render_template('index.html', message=message)
    else:
        message = 'Wrong password!'
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    user = {"name":"Ellie"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM top_shopping"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['Types'] = i[1]
        dict['Phone'] = i[2]
        dict['Address'] = i[3]
        dict['Price'] = i[4]
        dict['Website'] = i[5]
        dict['Hours'] = i[6]
        dict['Picture_url'] = i[7]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    
    return render_template('shopping.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'Wrong password!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########    

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "1155129577@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Ellie's pages. Your login has been granted."
        return jsonify({'response':'ok!','message':message})
    else:
        message = 'Wrong password!'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})
    
@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    user = {"name":"Ellie"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM top_shopping"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['Types'] = i[1]
        dict['Phone'] = i[2]
        dict['Address'] = i[3]
        dict['Price'] = i[4]
        dict['Website'] = i[5]
        dict['Hours'] = i[6]
        dict['Picture_url'] = i[7]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return jsonify({'Album': dataset})

######### Run Flask Web App at Port 9010 ##########    

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9010, app)

 * Running on http://localhost:9010/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2020 12:25:40] "GET / HTTP/1.1" 200 -
[2020-03-09 12:25:41,939] ERROR in app: Exception on /album [GET]
Traceback (most recent call last):
  File "/Users/jindanyang/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jindanyang/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jindanyang/opt/anaconda3/lib/python3.7/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/jindanyang/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/jindanyang/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value